In [11]:
from selenium import webdriver
import time
# time.sleep(1) 사용하기 위해 1초동안 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

# 제어 브라우저 작동(윈도우)
# driver=webdriver.Chrome("chromedriver")
# time.sleep(1)

#제어 브라우저 작동(리눅스)
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
time.sleep(1)

In [12]:
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver= webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)
#yield driver
time.sleep(1)


In [13]:
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")
time.sleep(1)

#대통령선거 id, click
driver.find_element_by_id("electionType1").click()
time.sleep(1)

# 제19대 선택 id, send_keys
driver.find_element_by_id("electionName").send_keys("제19대")
time.sleep(1)

# 대통령선거 선택 id, send_keys
driver.find_element_by_id("electionCode").send_keys("대통령선거")
time.sleep(1)

# 시도이름 선택 select id
sido_list_raw = driver.find_element_by_id("cityCode")
time.sleep(1)

# 시도이름 선택 option tag
sido_list = sido_list_raw.find_elements_by_tag_name("option")
time.sleep(1)

In [14]:
#시도이름 추출
sido_names_vales=[op.text for op in sido_list]
sido_names_vales=sido_names_vales[2:] # 합계, 선택 제외
sido_names_vales

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [15]:
election_result_raw = {'광역시도':[],
                       '시군':[],
                       'pop':[],
                       'moon':[],
                       'hong':[],
                       'ahn':[]
                      }

In [16]:
import re
def get_num(tmp):
    return re.split('\(', tmp)[0].replace(",", "") #

In [17]:
for each_sido in sido_names_vales:
    
    # 시도이름 선택 select id
    element = driver.find_element_by_id("cityCode")
    element.send_keys(each_sido)

    #검색버튼 클릭 click
    driver.find_element_by_id("searchBtn").click()

    #     #시도코드 선택 send_keys
    #     driver.find_element_by_id("cityCode").send_keys(each_sido)

    # 소스코드 전체파싱
    html = driver.page_source

    # 소스코드 받아오기
    soup = BeautifulSoup(html, "html.parser")

    # 테이블 soup에 넣기
    table = soup.find(id="table01")

    # df 문자열로 바뀐 상태에서 문자열만 추출해 내어 데이터프레임형태로 list에 저장
    df=pd.read_html(str(table))
    
    #print(f'df[0].values[1:]:{df[0].values[1:]}')
    
    for each in df[0].values[1:]:
        election_result_raw['광역시도'].append(each_sido)
        election_result_raw['시군'].append(each[0])
        election_result_raw['pop'].append(each[2])  # popular 인구수
        election_result_raw['moon'].append(get_num(each[3]))
        election_result_raw['hong'].append(get_num(each[4]))
        election_result_raw['ahn'].append(get_num(each[5]))

In [18]:
columns = []
for key in election_result_raw.keys():
    columns.append(key)
columns

['광역시도', '시군', 'pop', 'moon', 'hong', 'ahn']

In [9]:
driver.quit()